In [ ]:
import re
import openpyxl
import os
import DNS
from tkinter import Tk
from tkinter.filedialog import askopenfilename






In [ ]:
def clean_data(file_path):
    data = []
    workbook = openpyxl.load_workbook(file_path)
    sheet = workbook[workbook.sheetnames[0]]
    header_row = None
    email_columns = ['Email 1', 'Email 2', 'Email 3']
    phone_columns = ['Main Phone', 'Cell 2', 'Cell 3']

    for i, row in enumerate(sheet.rows):
        if i == 0:
            header_row = [cell.value for cell in row]
            email_column_indices = [header_row.index(col) for col in email_columns if col in header_row]
            phone_column_indices = [header_row.index(col) for col in phone_columns if col in header_row]
            data.append(header_row)
            continue

        include_row = True
        row_data = [cell.value for cell in row]

        # Validate email addresses
        for col_index in email_column_indices:
            email_address = row_data[col_index]
            if email_address:
                email_parts = email_address.split("@")
                if len(email_parts) != 2:
                    include_row = False
                    break
                domain = email_parts[1]
                try:
                    answers = DNS.resolver.query(domain, 'MX')
                except:
                    include_row = False
                    break

        # Remove phone numbers starting with "25420" or "20"
        for col_index in phone_column_indices:
            phone_number = row_data[col_index]
            if phone_number and str(phone_number.startswith("25420") or phone_number.startswith("20")):
                row_data[col_index] = None

        if include_row:
            data.append(row_data)

    return data

In [ ]:
def write_to_xlsx(data, file_path):
    workbook = openpyxl.Workbook()
    sheet = workbook.active
    for row in data:
        sheet.append(row)
    workbook.save(file_path)


In [ ]:
root = Tk()
root.withdraw()
file_path = askopenfilename(filetypes=[("Excel Files", "*.xlsx")])
cleaned_data = clean_data(file_path)
directory = os.path.dirname(file_path)
file_name = os.path.basename(file_path)
new_file_path = os.path.join(directory, "clean_" + file_name)
write_to_xlsx(cleaned_data, new_file_path)
